In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
from ngram_tools.download_ngrams import download_ngram_files
from ngram_tools.convert_to_jsonl import convert_to_jsonl_files
from ngram_tools.lowercase_ngrams import lowercase_ngrams
from ngram_tools.lemmatize_ngrams import lemmatize_ngrams
from ngram_tools.filter_ngrams import filter_ngrams
from ngram_tools.sort_ngrams import sort_ngrams
from ngram_tools.consolidate_ngrams import consolidate_duplicate_ngrams
from ngram_tools.make_yearly_files import make_yearly_files
from ngram_tools.helpers.verify_sort import check_file_sorted
from ngram_tools.helpers.print_jsonl_lines import print_jsonl_lines

# **Process Multigrams for Training Word-Embedding Models**

## **Goal**: Download and preprocess mulitgrams for use in training `word2vec` models. 

This workflow is resource-intensive and is probably only practical when run on a computing cluster. On my university's High Performance Computing (HPC) cluster, I request the maximum 14 cores (48 logical processors) and 128G of memory and use a 2T fast-I/O NVMe SSD filespace—and I still run up against time and resource limits. I've designed the code to be efficient, although further optimization is surely possible.

The code affords options to conserve resources. Throughout the workflow you can specify `compress=True`, which tells a script to compress its output files. In my experience, there is little downside to using LZ4 compression, since it's very fast and cuts file sizes by about half. Downstream modules will see the `.lz4` extensions and handle the files accordingly. If you know your workflow runs correctly and wish to further conserve space, you can specify `delete_input=True` for many of the scripts; this will delete the source files for a given step once it is complete. The scripts are fairly memory-efficient—with the exception of `sort_ngrams` and `index_and_create_vocab_files`, which sort multiple files in memory at once. When processing multigrams, I've found that allocating more than ~10 workers in these scripts leads to memory exhaustion (with 128G!) and slow processing.

**NOTE:** You'll probably want to have run `workflow_unigrams.ipynb` before processing multigrams. That workflos allows you create a vocabulary file for filtering out uncommon tokens from the multigrams. Although you can run the `filter_ngrams` module without a vocab file, most use cases will call for one.

### Download multigrams
Here, I'm using `download_ngrams` module to fetch 5grams appended with part-of-speech (POS) tags (e.g., `_VERB`). Although you can specify `ngram_type='untagged'`, POS tags are necessary to lemmatize the tokens. Specify the number of parallel processes you wish to use by setting `workers` (the default is all available processors). You may wish to specify `compress=True` becausae 5gram files are _big_.

In [3]:
download_ngram_files(
    ngram_size=5,
    ngram_type='tagged',
    repo_release_id='20200217',
    repo_corpus_id='eng-us',
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-us',
    compress=True,
    overwrite=True
)

Start Time:                2025-04-06 17:32:40.329758

Download Info
Ngram repository:          https://storage.googleapis.com/books/ngrams/books/20200217/eng-us/eng-us-5-ngrams_exports.html
Output directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/5gram_files/1download
File index range:          0 to 11144
File URLs available:       11145
File URLs to use:          11145
First file to get:         https://storage.googleapis.com/books/ngrams/books/20200217/eng-us/5-00000-of-11145.gz
Last file to get:          https://storage.googleapis.com/books/ngrams/books/20200217/eng-us/5-11144-of-11145.gz
Ngram size:                5
Ngram type:                tagged
Number of workers:         48
Compress saved files:      True
Overwrite existing files:  True



Downloading:   0%|          | 0/11145 [00:00<?, ?files/s]


End Time:                  2025-04-06 20:11:13.102248
Total runtime:             2:38:32.772490



### Convert files from TXT to JSONL
This module converts the original multigram files' text data to a more flexible JSON Lines (JSONL) format. Although this increases storage demands, it makes downstream processing more efficient.

In [4]:
convert_to_jsonl_files(
    ngram_size=5,
    ngram_type='tagged',
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-us',
    compress=True,
    overwrite=True,
    delete_input=True
)

Start Time:                2025-04-06 20:11:13.195440

Conversion Info
Input directory:           /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/5gram_files/1download
Output directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/5gram_files/2convert
File index range:          0 to 11144
Files available:           11145
Files to use:              11145
First file to get:         /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/5gram_files/1download/5-00000-of-11145.txt.lz4
Last file to get:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/5gram_files/1download/5-11144-of-11145.txt.lz4
Ngram size:                5
Ngram type:                tagged
Number of workers:         48
Compress output files:     True
Overwrite existing files:  True
Delete input directory:    True



Converting:   0%|          | 0/11145 [00:00<?, ?files/s]


End Time:                  2025-04-06 20:59:52.674704
Total runtime:             0:48:39.479264



### Make multigrams all lowercase
This module lowercases all characters in the multigrams. Most use cases benefit from this.

In [5]:
lowercase_ngrams(
    ngram_size=5,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-us',
    compress=True,
    overwrite=True,
    delete_input=True
)

Start Time:                2025-04-06 20:59:52.762542

Lowercasing Info
Input directory:           /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/5gram_files/2convert
Output directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/5gram_files/3lowercase
File index range:          0 to 4006
Files available:           4007
Files to use:              4007
First file to get:         /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/5gram_files/2convert/5-00000-of-11145.jsonl.lz4
Last file to get:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/5gram_files/2convert/5-11144-of-11145.jsonl.lz4
Ngram size:                5
Number of workers:         48
Compress output files:     True
Overwrite existing files:  True
Delete input directory:    True



Lowercasing:   0%|          | 0/4007 [00:00<?, ?files/s]


End Time:                  2025-04-06 21:24:11.173712
Total runtime:             0:24:18.411170



### Lemmatize the multigrams
Likewise, most use cases will benefit from multigrams that are lemmatized—that is, reduced to their base form. This requires POS-tagged multigrams. Example: `people_NOUN` ("the people of this land") will be converted to `person` in the output; `people_VERB` ("to people this land") will not. The POS tag will then be discarded as it is no longer useful.

In [ ]:
lemmatize_ngrams(
    ngram_size=5,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-us',
    compress=True,
    overwrite=True,
    delete_input=True
)

Start Time:                2025-04-06 22:17:12.029756

Lemmatizing Info
Input directory:           /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/5gram_files/3lowercase
Output directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/5gram_files/4lemmatize
File index range:          0 to 4006
Files available:           4007
Files to use:              4007
First file to get:         /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/5gram_files/3lowercase/5-00000-of-11145.jsonl.lz4
Last file to get:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/5gram_files/3lowercase/5-11144-of-11145.jsonl.lz4
Ngram size:                5
Number of workers:         48
Compress output files:     True
Overwrite existing files:  True
Delete input directory:    True



Lemmatizing:   0%|          | 0/4007 [00:00<?, ?files/s]

### Filter the multigrams
This module removes tokens that provide little information about words' semantic context—specifically, those that contain numerals (`numerals=True`), nonalphabetic characters (`nonalpha=True`), stopwords (high-frequency, low information tokens like "the" and "into"; `stops=True`), or short words (those below a certain user-specified character count; here, `min_token_length=3`). You can also specify a **vocabulary file** like the one produced in the unigram workflow. A vocabulary file is simply a list of the _N_ most common words in the unigram corpus; the multigram tokens are checked against this list and those that don't appear in it are dropped.

The `replace` option controls what happens to ineligible tokens. If `replace=False` (the default), then these tokens are erased from the ngrams. If `replace=True`, then ineligible tokens will be replaced with `UNK` (the "unknown" symbol). The filtering process will inevitably reduce the amount of useful information contained in some ngrams. If `replace=False`, some longer ngrams (e.g., 5grams) will become shorter (e.g., 3grams) after unwanted tokens are dropped. And if `replace=True`, filtering will reduce the number of real tokens in certain ngrams by replacing one or more of tokens with `UNK`.

The training of word-embedding models requires _linguistic context_, which in turn requires ngrams containing more than one token. (A unigram isn't useful for helping a model learn what "company" a word keeps.) The `min_tokens` option allows you to drop ngrams that fall below a specified length (or number of real tokens) during filtering. If filtering results in an ngram with fewer than the minimum number of real tokens, all data for that ngram is dropped entirely. I usually set `min_tokens=2`, since two tokens (and higher) provide at least some contextual information.

In [7]:
filter_ngrams(
    ngram_size=5,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-us',
    numerals=True,
    nonalpha=True,
    stops=True,
    min_token_length=3,
    min_tokens=2,
    vocab_file='1gram-corpus-vocab_list_match.txt',
    compress=False,
    overwrite=True,
    delete_input=False
)

Start Time:                   2025-02-22 02:51:36.373929

Filtering Info
Input directory:              /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/5gram_files/4lemmatize
Output directory:             /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/5gram_files/5filter
File index range:             0 to 4006
Files available:              4007
Files to use:                 4007
First file to get:            /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/5gram_files/4lemmatize/5-00000-of-11145.jsonl.lz4
Last file to get:             /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/5gram_files/4lemmatize/5-11144-of-11145.jsonl.lz4
Ngram size:                   5
Number of workers:            48
Compress output files:        True
Overwrite existing files:     True
Delete input directory:       True

Filtering Options
Drop stopwords:               True
Drop tokens under:            3 chars
Drop tokens with numerals:    True
Drop non-alphabetic:          True
Drop n

Filtering:   0%|          | 0/4007 [00:00<?, ?files/s]


Filtering Results (Dropped)
Stopword tokens:              0 
Short-word tokens:            0 
Tokens with numerals:         0 
Tokens with non-alpha chars:  0
Out-of-vocab tokens:          3986732180
Entire ngrams:                298745980 

End Time:                  2025-02-22 03:18:22.317299
Total runtime:             0:26:45.943370



### Sort and combine the multigram files
This modules creates a single, fully-sorted multigram file out of the filtered files. This is crucial for the next step (ngram consolidation; see below).   

Sorting a giant file is a resource-hungry process and I've tried to implement an efficient approach that leverages parallelism: We first sort the filtered files in parallel using Python's standard sorting algorithm [Timsort](https://en.wikipedia.org/wiki/Timsort); then, we incrementally [heapsort](https://en.wikipedia.org/wiki/Heapsort) the files in parallel until we get down to 2 files. Finally, we heapsort the final 2 files (necessarily using one processor) to arrive at a single combined and sorted unigram file.

Because this step can take a _very_ long time for larger multigrams (e.g., 5grams), we can run it in sessions using the `start_iteration` and `end_iteration` options. Iteration 1 comes after the initial file sort. If you only have time to complete, say, iterations 1–3, you can set `end_iteration=3`. During a later session, you can specify `start_iteration=4` to pick up where you left off.

In [ ]:
sort_ngrams(
    ngram_size=5,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-us',
    workers=12,
    sort_key='ngram',
    compress=False,
    overwrite=True,
    sort_order='ascending',
    start_iteration=6,
    delete_input=False
)

Start Time:                2025-02-22 17:27:28.012897

Sort Info
Input directory:           /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/5gram_files/5filter
Sorted directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/5gram_files/temp
Temp directory:            /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/5gram_files/tmp
Merged file:               /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/5gram_files/6corpus/5gram-merged.jsonl.lz4
Files available:           4006
First file to get:         /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/5gram_files/5filter/5-00043-of-11145.jsonl.lz4
Last file to get:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/5gram_files/5filter/5-11144-of-11145.jsonl.lz4
Files to use:              4006
Ngram size:                5
Number of workers:         12
Compress output files:     True
Overwrite existing files:  False
Sort key:                  ngram
Sort order:                ascending
H

Sorting:   0%|          | 0/4006 [00:00<?, ?files/s]


Iteration 6: merging 3 files into 1 chunks using 1 workers.
  1 chunk(s) with 3 file(s)


### Verify sort [OPTIONAL]
If we want, we can verify that the output file is correctly sorted. If the script outputs True, then the file is sorted. Bear in mind that you need to specify the file path manually here; be sure to use the right file extension based on whether sort_ngrams was run with `compress=True`.

In [ ]:
check_file_sorted(
    input_file=(
        '/vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/'
        '5gram_files/6corpus/5gram-merged.jsonl.lz4'
    ),
    field="ngram",
    sort_order="ascending"
)

Lines: 644454086line [1:32:23, 113542.17line/s]

### Consolidate duplicate multigrams
This module consolidates the sorted multigram file. Lowercasing and lemmatizing produce duplicate unigrams. Now that the file is sorted, we can scan through it and consolidate consecutive idential duplicates. This involves summing their overall and yearly frequencies and document counts. It also leads to a much smaller file.

In [ ]:
consolidate_duplicate_ngrams(
    ngram_size=5,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-us',
    lines_per_chunk=500000,
    compress=False,
    overwrite=True
)

Start Time:                2025-02-23 12:59:39.240776

Consolidation Info
Merged file:               /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/5gram_files/6corpus/5gram-merged.jsonl.lz4
Corpus file:               /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/5gram_files/6corpus/5gram-corpus.jsonl.lz4
Temporary directory:       /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/5gram_files/temp_chunks
Ngram size:                5
Number of workers:         48
Compress output files:     True
Overwrite existing files:  True

Created and Sorted: 2185 chunks


### View line [OPTIONAL]
If we want, we can inspect a line in the file.

In [6]:
print_jsonl_lines(
    file_path=(
        '/vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/'
        '5gram_files/6corpus/5gram-corpus.jsonl.lz4'
    ),
    start_line=1650262,
    end_line=1650263,
    parse_json=True
)

Line 1650262: {'ngram': 'tube superimpose', 'freq_tot': 52, 'doc_tot': 51, 'freq': {'1939': 1, '1942': 1, '1949': 2, '1951': 1, '1952': 3, '1955': 4, '1957': 1, '1958': 2, '1961': 2, '1964': 1, '1967': 1, '1968': 1, '1969': 1, '1970': 6, '1971': 1, '1972': 1, '1973': 2, '1974': 1, '1975': 3, '1976': 1, '1977': 2, '1978': 1, '1979': 2, '1980': 1, '1981': 2, '1982': 2, '1983': 5, '1987': 1}, 'doc': {'1939': 1, '1942': 1, '1949': 2, '1951': 1, '1952': 3, '1955': 3, '1957': 1, '1958': 2, '1961': 2, '1964': 1, '1967': 1, '1968': 1, '1969': 1, '1970': 6, '1971': 1, '1972': 1, '1973': 2, '1974': 1, '1975': 3, '1976': 1, '1977': 2, '1978': 1, '1979': 2, '1980': 1, '1981': 2, '1982': 2, '1983': 5, '1987': 1}}
Line 1650263: {'ngram': 'tube superimpose glass', 'freq_tot': 49, 'doc_tot': 48, 'freq': {'1939': 1, '1942': 1, '1949': 2, '1951': 1, '1952': 3, '1955': 4, '1957': 1, '1958': 2, '1961': 2, '1964': 1, '1967': 1, '1968': 1, '1969': 1, '1970': 5, '1972': 1, '1973': 2, '1974': 1, '1975': 2, '1

### Make yearly files
This module converts the overall corpus file into yearly corpora. For each year in which an ngram appeared, a `<year>.jsonl` file (or `<year>.jsonl.lz4` if `compress=True`) will be created. Each line in a yearly file contains an ngram, a `freq` value (the number of times it appeared that year), and a `doc` value (the number of unique documents it appeared in that year).

I found it difficult to prevent memory exhaustion when processing 5grams with 128GB of RAM. Users may have to reduce the number of processors and/or the `chunk_size` to stay within their limits. Also note that the final clean-up step, in which many temporary files get deleted, can take several minutes to complete. 

After creating yearly corpora, we can proceed to train `word2vec` models as shown in the `workflow_train_models.ipynb` notebook.

In [2]:
make_yearly_files(
    ngram_size=5,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-us',
    overwrite=True,
    compress=False,
    workers=14,
    chunk_size=500000
)

Start Time:                2025-02-24 09:35:48.740053

Processing Info
Corpus file:               /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/5gram_files/6corpus/5gram-corpus.jsonl.lz4
Yearly file directory:     /vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/5gram_files/6corpus/yearly_files/data
Compress output files:     True
Number of workers:         14
Overwrite existing files:  True

Created and processed 468 chunks
Merged temp files for 482 years

End Time:                  2025-02-24 10:49:29.883208
Total runtime:             1:13:41.143155



In [3]:
print_jsonl_lines(
    file_path=(
        '/vast/edk202/NLP_corpora/Google_Books/20200217/eng-us/'
        '5gram_files/6corpus/yearly_files/data/2019.jsonl.lz4'
    ),
    start_line=1650262,
    end_line=1650263,
    parse_json=True
)

Line 1650262: {'ngram': 'air force cut', 'freq': 2, 'doc': 2}
Line 1650263: {'ngram': 'air force daf', 'freq': 3, 'doc': 2}


### Next Steps
Now that you've created yearly corpora of multigrams, it's time to train word embeddings using `word2vec`. See the `workflow_train_models.ipynb` notebook for a guide to training and optimizing yearly word embeddings.